## Load submission IDs from hacks

In [ ]:
submissionIDs = set()
submissionToHack = dict()
f = open("submissionUrls.txt").read().splitlines()
for line in f:
    subID = line.split()[-1].split("/")[-1]
    submissionIDs.add(int(subID))
    submissionToHack[int(subID)] = int(line.split()[0])
len(submissionIDs)

## Connect to database

In [ ]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1',
                              database='code4bench')
print(cnx)

## Iteratively match hacks with submission ids

In [ ]:
mycursor = cnx.cursor()
matchingSubmission = []
for step in range(35):
#step = 0
    query = f"SELECT * FROM source where id < {100000*(step+1)} and id > {100000*step}"
    
    mycursor.execute(query)

    myresult = mycursor.fetchall()
    for x in myresult:
        if x[1] in submissionIDs:
            matchingSubmission.append(x)
    print (step, len(matchingSubmission))

## Get Programming language details

In [ ]:
pls = [(1,'GNU C++14'),(2,'GNU C'),(3,'MS C++'),(4,'GNU C++11'),(5,'FPC'),(6,'GNU C++'),(7,'Java 8'),(8,'Python 3'),(9,'Go'),(10,'D'),(11,'MS C#'),(12,'GNU C11'),(13,'Python 2'),(14,'PyPy 2'),(15,'Ruby'),(16,'PHP'),(17,'PyPy 3'),(18,'Delphi'),(19,'Kotlin'),(20,'JavaScript'),(21,'Haskell'),(22,'Ocaml'),(23,'Scala'),(24,'Mono C#'),(25,'Java 7'),(26,'Rust'),(27,'Perl'),(28,'GNU C++11 ZIP'),(29,'Java 8 ZIP'),(30,'J'),(31,'GNU C++0x'),(32,'Java 6'),(33,'Pike'),(34,'Befunge'),(35,'Cobol'),(36,'Factor'),(37,'Secret_171'),(38,'Roco'),(39,'Tcl'),(40,'F#'),(41,'Io')]
plDict = {a:b for a,b in pls}

In [ ]:
data = []
for entry in matchingSubmission:
    submissionId = int(entry[1])
    hackId = submissionToHack[submissionId]
    sourceCode = entry[2]
    pl = plDict[entry[-2]]
    problemID = entry[-4]
    data.append({"submissionID":submissionId,"hackId":hackId,"sourceCode":sourceCode, "programmingLanguage":pl,"problemID":problemID})

## Store data in json file

In [ ]:
import json 
with open("submissionCodeNEW.json", "w") as final:
    json.dump(data, final)

# Match test cases with problem ID

In [ ]:
import time
for entry in data:
    testcases = []
    p_id = entry["problemID"]
    query = f"SELECT * FROM testcases where problems_id = {p_id}"
    mycursor.execute(query)

    myresult = mycursor.fetchall()
    for t in myresult:
        # only include complete tests
        if "..." in t[1] or "..." in t[2]:
            continue
        testcases.append({"id": t[0],"input":t[1],"output":t[2]})
        
        
    with open(f"testcases/problem-{p_id}.json", "w") as f:
        json.dump(testcases, f)